## 1. Import

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

## 2. 데이터 가져오기

In [2]:
train = pd.read_csv('creditcard.csv')

## 3. 데이터 확인

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
train.shape

(284807, 31)

## 4. 데이터 분할

In [5]:
X = train.iloc[:,:-1].values
y = train.iloc[:,-1].values

# Encoding the dependent variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

## 5. 모델링

### 5-1. LightGBM

In [6]:
lgb_clf = lgb.LGBMClassifier(objective='binary',
                             learning_rate=0.1,
                             max_depth=16,
                             min_data_in_leaf=200,
                             num_leaves=500,
                             boosting='gbdt'
                            )

lgb_clf.fit(X_train, y_train)

y_pred = lgb_clf.predict(X_test)

print('f1 score:', f1_score(y_test, y_pred))

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
f1 score: 0.9017341040462428


LightGBM의 특징

장점
- 학습하는데 걸리는 시간이 짧다.
- 메모리 사용량이 상대적으로 적다.
- 대용량 데이터를 처리 가능하다.

단점

- 적은 데이터셋(10,000건 이하)에서는 과적합이 발생 가능하다.

LightGBM 파라미터
- learning_rate: 부스팅을 반복할 때의 학습률로, 0~1 사이의 값
- max_depth: 트리의 깊이
- min_data_in_leaf: 한 리프의 최소 데이터 수
- num_leaves: 하나의 트리가 가질 수 있는 최대 리프 개수
- boosting: 부스팅 방법
- objective: 수치예측이면 regression, 이진분류이면 binary

### 5-2. XGBoost

In [7]:
xgb_clf = xgb.XGBClassifier(max_depth=3, 
                            n_estimators=500, 
                            learning_rate=0.1,
                            min_child_weight=3,
                            gamma=0.2
                           )

xgb_clf.fit(X_train,y_train)

y_pred = xgb_clf.predict(X_test)

print('f1 score:', f1_score(y_test, y_pred))

f1 score: 0.9017341040462428


XGBoost의 특징

- GBM 대비 빠른 수행 시간: 병렬 처리로 학습, 분류 속도가 빠르다.
- 과적합 규제(Regularization): 표준 GBM의 경우 과적합 규제기능이 없으나, XGBoost는 자체에 과적합 규제 기능으로 강한 내구성이 있다.
- 분류와 회귀영역에서 예측 성능이 뛰어나다.
- Early Stopping(조기 종료) 기능이 있다.
- 다양한 옵션을 제공하며 Customizing이 용이하다.
- Missing Values : 결측치를 내부적으로 처리해준다.

XGBoost 파라미터
- max_depth: 트리의 최대 깊이
- n_estimators: 트리 모델의 개수
- learning_rate: 학습 단계별로 이전 결과를 얼마나 반영할지 설정
- min_child_weight: child 에서 필요한 모든 관측치에 대한 가중치의 최소합
- gamma: 트리에서 추가적으로 가지를 나눌지 말지 결정하는 최소 손실 감소 값, 값을 크게 설정할 수록 과적합이 감소하는 효과가 있다.

### 5-3. Decision Tree

In [9]:
dt_clf = DecisionTreeClassifier(max_depth=5,
                               )

dt_clf.fit(X_train, y_train)

y_pred = dt_clf.predict(X_test)

print('f1_score:', f1_score(y_pred,y_test))

f1_score: 0.8587570621468926


Decision Tree의 특징

장점
- 데이터의 전처리 (정규화, 결측치, 이상치 등)를 하지 않아도 된다.
- 수치형과 범주형 변수를 한꺼번에 다룰 수 있다.

단점
- 과적합으로 알고리즘 성능이 떨어질 수 있다.

Decision Tree 파라미터
- max_depth: 트리의 최대 깊이